<h1>CS152 Assignment 2: The DPLL Algorithm</h1>

In this assignment, you are required to implement the DPLL algorithm in python as described in Figure 7.17 in Russell & Norvig, and demonstrate it on a given KB.

## Assignment Overview
This assignment consists of several parts that should be completed in python code. You may use a
python notebook with markdown cells for documentation, or a python script with substantial commenting to explain your code.

1. Implement a python class Literal, which will be used to represent a logical literal. This class
should contain a name attribute, which is a string containing the name of the literal. In addition, since clauses in DPLL may include negated literals, the class should also have a sign
attribute, which is True if the literal is positive and False if the literal is negative. The class
should also overload the `__neg__(self,other)` method to allow negative literals to be defined in clauses. As an example, the class should allow us to define literals like:

    ```
    - A = Literal(A)
    - B = Literal(B)
    - C = Literal(C)
    - D = Literal(D)
    ```

   which will then allow us to define a KB in CNF comprising clauses by a list of sets. For example, if we wish to represent the knowledge base given by the CNF
    
    $$(A ∨ B) ∧ (A ∨ ¬C) ∧ (¬A ∨ B ∨ D),$$

   we would write the python code
   
   ```
   KB = [{A,B},{A,-C},{-A,B,D}]
   ```
   
   Each set indicates a disjunction of the literals contained within it, with the negative sign indicating a negated literal.
   
   
2. Implement a simplified version of the DPLL algorithm as in Figure 7.17, excluding the pure symbol and unit clause heuristics. This essentially reduces to choosing an arbitrary unassigned literal and trying to assign both true and false values to it and performing recursive evaluation of DPLL. You should have a function that is callable using the signature
    
    ```
   (satisfiable, model) = DPLLSatisfiable(KB)
   ```
   
   where KB is the knowledge base as above. This contains an additional output to the specification in Russell & Norvig. The outputs are satisfiable - a boolean value indicating whether or not the KB is satisfiable, and model - a python dictionary that has a Literal as the key and the value being one of the strings ‘true’, ‘false’ or ‘free’, depending on whether the literal must be true or false or can be freely chosen to ensure satisfiability. Remember that this function only has to return one model that works (even though there may be several). An example output for the KB above would be
   
   ```
   model = {A: true, B: true, C: false, D: free}
   ```
   
   Note that in this case, the literal A could also be free, but depending on the DPLL trace, A may be assigned in a model. The free variables will in practice be those symbols left over after DPLL has run to completion.
   
3. Test your code on the KB given in Exercise 7.20 of Russell & Norvig. Convert the KB into CNF and then run DPLL to determine a model for the KB and print it out.

### Extensions
1. Instead of choosing a symbol to assign arbitrarily, choose the one that is yet to be assigned
and occurs in the most number of clauses at each point in the algorithm (known as the degree
heuristic)

2. Implement the pure symbol and unit clause heuristics, which are the primary innovations of
DPLL over a simple recursive depth-first enumeration over true-false values. Extra kudos
will be given for choosing the pure symbol that occurs the most over all clauses each time
rather than choosing one arbitrarily.

### Hints
1. You can assume that a literal can occur at most once in every clause, either non-negated or
negated. Two literals of the same sign in the one clause would cancel out and reduce to a
single literal $(A ∨ A = A)$, and two literals of opposing signs will make the clause trivially
true i.e. $(A ∨ ¬A = true)$.

2. A clause is true as soon as any of its literals are true, and a clause with any literals false can
be replaced by a clause with those literals removed. The only way to falsify a clause is if all
literals (taking signs into account) are false.

3. You may wish to put literals into a set regardless of their sign and not treat two literals with
the same name, but different signs, as distinct. In this case, you can overload the equality
operator `__eq__(self,other)` in order to specify that two literals are equal regardless of their
sign (i.e. based only on their names). An alternative would be to use two sets for each clause,
one storing the positive literals and one storing the negative ones.

4. You may find it useful to define literals ‘true’ and ‘false’, and use these to indicate whether
a clause is true in a given model or has been falsified. Additionally, at the execution point where all of the clauses are true, you could save the model using a global variable, that you
can read once the DPLL recursion has terminated.

---

<h1>Question 1</h1>

Define your <code>Literal</code> class below.  Ensure that you define a <code>name</code> and <code>sign</code> attribute as required in the assignment description.  In addition, include any other attributes and member functions as needed. You will need to overload <code>__neg__()</code> to correctly handle negated literals.  You may need to overload <code>__eq__()</code> and <code>__hash__()</code> also.

In [1]:
# Import any packages you need here
from functools import reduce
from copy import deepcopy
from collections import Counter

class Literal:
    
    def __init__(self,name):
        
        """
        Literal class to represent positive and negative literals.
    
        Inputs/Attributes:
            - name: Name of the literal
            - sign: Sign to indicate if literal is positive or negative.
                    Set to True for positive literals and False for negative literals
    
        """
        
        #If the Literal has a negation symbol, assign False to sign, else assign True
        if name[0] in ('-', '!','￢', '~'): 
            self.name= name[1:]
            self.sign= False
        else:
            self.name = name
            self.sign = True
        
    def __neg__(self):
        
        #Overload negation to alter the sign of a Literal
        
        if self.sign:
            return Literal('{}{}'.format('-',self.name))
        else:
            return Literal(self.name)
        
    def __eq__(self, other):
        
        #Overloading to ensure that two literals with opposite sign are equal (i.e. same name)
        
        return self.name == other.name
    
    
    def __lt__(self, other):
        
        #Overload to allow alphabetical sorting based on names.
        return self.name < other.name
    

    def __hash__(self):
        return hash(self.name)
    
    def __str__(self):
        return self.name if self.sign else '￢{}'.format(self.name)

    def __repr__(self):
        return self.name    
    
    def standardize(self):
        return Literal(self.name)
        

<h1>Question 2</h1>
Implement DPLL by filling in the stubs below.  Note that the various heuristics are not required to be implemented for basic functionality, but a template has been provided for you to do so if you attempt the extension questions

In [2]:
# Define degree, pure symbol and unit clause heuristics here (optional).
# Add in your functions under the templates below

def true_clause(clause,model):
    
    """
    Checks if the inputted clause is True based on the literal value assignment
    in the inputted model. Once one of the literals is found to be True, the clause
    is determined to be True as well. This follows from the  fact that a clause is a 
    disjunction of literals.
    
    Inputs:
        - clause (set): A set of Literal objects
        - model (dict): A dictionary specifying the Literal assignment (True or False)
                        in the given model. The keys are Literal objects and the 
                        value of each key is either True or False.
        
    Output:
        - bool: Returns True if the clause is True in the given model. Returns False otherwise.
        - None: Returns None if the truth status of the clause cannot be determined from the model
    
    """
    
    #Placeholder to track symbols with unknown truth values (unassigned symbols)
    unknown= False
    
    #Check each Literal in the clause
    for literal in clause:
        
        
        #If the literal has not been assigned, set unknown to True to indicate that the
        #truth value cannot be determined
        if literal not in model:
            if unknown is False:
                unknown= True
        
        #If the literal is assigned in the model, and evaluates to True after assignment
        elif literal in model and literal.sign == model[literal]:
            
            #Return True, indicating that the clause is True in the model
            return True
    
    #If the truth value of a clause cannot be determined, return None
    if unknown:
        return None
    #If all the symbols in the clause evaluate to False after assignment, return False
    else:
        return False

def update(model, key, value):
    
    """
    Copy and update model by setting key to value. Return the copy with new update.
    
    Inputs:
        - model (dict): Dictionary containing Literal assignments. The keys of the dictionary 
                        represent the Literals and the values of each represents their assignment 
                        in the model(either True or False)
        - key (Literal): The Literal to be added to the model
        - value (bool): True or False assignment to key in the given model
        
    Output:
        - updated_model (dict): Model with added Literal and its assignment
    
    
    """
    
    updated_model = model.copy()
    updated_model[key] = value

    return updated_model

def degree_heuristic(symbols,KB,model):
    
    
    """
    Order symbols in terms of their frequency of occurrence in the KB and return the symbol that 
    occurs most frequently (and is currently unassigned) as well as the other symbols in the KB. 
    Symbols with same frequency of occurence are ordered alphabetically.
    
    Inputs:
        - symbols: A set containing all propositional symbols the KB
        - KB: Knowledge base in the form of a list of sets of Literal objects, where each set
              represents a clause in CNF form. 
        - model(dict): The configuration of the world (assignment of literals). The keys of the 
                       dictionary represent the Literals and the values of each represents their assignment 
                       in the model (either True or False).
                       
    Outputs:
        - symbol: An unassigned symbol with the highest frequency in the KB
        - rest: A list containing all other symbols in the KB (after exclusion of symbol described above)
    
    """
    
    
    #Flatten knowledge base to get every single symbol in the KB
    flat_KB = [symbol for clause in KB for symbol in clause]
    
    #Count the occurrence of each of the symbol (Note: a postive and negative literal of the same symbol
    #are considered the same symbol). Here the output is a dictionary like structure with the symbol 
    #as the key and its frequency of occurrence as the corresponding value.
    symb_freq= Counter(flat_KB)
    
    #Sort symbols first based on decreasing frequency of occurrence then alphabetically
    freq_ord_symb = sorted(symb_freq, key=lambda x: (-symb_freq[x], -x))
    
    #Iterate through all the symbols after they have been ordered 
    for symbol in freq_ord_symb:
        
        #Return the most frequently occuring symbol that is currently not assigned in the model
        if symbol not in model:
            
            #Get the rest of the symbols after excluding the most frequently occuring symbol
            freq_ord_symb.remove(symbol)
            
            #Return the standardized symbol (get rid of negation if present) and the rest
            #of the symbols
            return symbol.standardize(), freq_ord_symb
        
def find_pure_symbol(symbols,KB,model,with_degree_heuristic= False):
    
    """
    
    Checks for pure symbols (symbols that appear only as a positive literal or only
    as a negative literal) in the KB clauses that are not already assigned in the model.
    In the event that there are multiple pure symbols, a pure symbol is returned if it comes before
    others alphabetically.
    
    Inputs:
        - symbols: A set containing all propositional symbols the KB
        - KB: Knowledge base in the form of a list of sets of Literal objects, where each set
              represents a clause in CNF form. 
        - model(dict): The configuration of the world (assignment of literals). The keys of the 
                       dictionary represent the Literals and the values of each represents their assignment 
                       in the model (either True or False).
    
    Output:
        - symbol, sign: Return a pure symbol and the sign the assignment that makes it True in 
                        all clauses. In the case of multiple pure symbols, ff with_degree_heuristic
                        is True the most frequently occuring pure symbol in the KB
                        is selected for assignment, else a symbol is selected alphabetically.
                        
        - None: Returns None if no pure symbol is found
    """
    
    #Replace all negative symbols with positive ones. This is needed to ensure the evaluation
    #of symbols to determine pure symbols on lines 160-164 is fair
    symbols = [symbol.standardize() if not symbol.sign else symbol for symbol in symbols] 

    
    #List to store the pure symbols found 
    pure_symbols= []
    
    #Flatten knowledge base to get every single symbol in the KB
    flat_KB = [symbol for clause in KB for symbol in clause]
    
    #For each symbol
    for symbol in symbols:
        
        #If the symbol is unassigned in the model
        if symbol not in model:
            
            #Create placeholders to track the valency of the symbol in all clauses
            pos_literal, neg_literal= False, False
            
            #For each symbol/literal in the knowledge base
            for literal in flat_KB:
                
                #Check the valency of each instance of the symbol under question 
                #and update placeholders accordingly
                if symbol == literal:
                    if (symbol.sign and literal.sign == True) and not pos_literal:
                        pos_literal= True
                    elif (-symbol.sign and literal.sign == False) and not neg_literal:
                        neg_literal = True
            
            #If the symbol is a pure symbol, add it to the list alongside the value that will make it
            #True in all clauses it is contained in
            if pos_literal != neg_literal:
                pure_symbols.append((symbol,pos_literal))
    
    #If there is at least one pure symbol
    if pure_symbols:
        
        #EXTENSION OF PURE SYMBOL HEURISTIC AS REQUIRED FOR HEURISTIC 3
        if with_degree_heuristic:
            
            #If there is just one pure symbol,return it and the assignment that makes it True in all clauses
            if len(pure_symbols)>1:
                
                #Count the occurrence of each of the symbol (here a positive and negative literal 
                #of the same symbol are considered the same symbol). Note: The output is a 
                #dictionary like structure with the symbol  as the key and its frequency of 
                #occurrence as the corresponding value.
                symb_freq= Counter(flat_KB)


                #Sort symbols first based on decreasing frequency of occurrence then alphabetically
                freq_ord_symb = sorted(symb_freq, key=lambda x: (-symb_freq[x], -x))


                #Iterate through all the symbols after they have been ordered 
                for symbol in freq_ord_symb:

                    #Return the most frequently occuring pure symbol that is 
                    #currently not assigned in the model
                    if (symbol,symbol.sign) in pure_symbols:

                        return symbol, symbol.sign
                    
            #If there is just one pure symbol,return it and the assignment that makes it True in all clauses
            else:
                return pure_symbols[0]
                
        
        #STANDARD IMPLEMENTATION OF PURE SYMBOL HEURISTIC
        else:
        
            #If there are multiple pure symbols
            if len(pure_symbols) > 1:
                

                #Return the symbol that comes first alphabetically and the assignment that makes it 
                #True in all clauses
                return sorted(pure_symbols,reverse=True)[0]

            #If there is just one pure symbol,return it and the assignment that makes it True in all clauses
            else:
                return pure_symbols[0]
     
    #If no pure symbol is found, return None
    else:
        return None, None
                      

def find_unit_clause(KB,model):
    
    """
    
    Returns a the unit clause from the remaining unproven clauses given model assignment
    
    Inputs:
        - KB: Knowledge base in the form of a list of sets of Literal objects, where each set
              represents a clause in CNF form. 
        - model(dict): The configuration of the world (assignment of literals). The keys of the 
                       dictionary represent the Literals and the values of each represents their assignment 
                       in the model (either True or False).
    
    Output:
        - symbol, sign: Return a pure symbol and the sign the assignment that makes it True in 
                        the unit clause. In the case of multiple symbols, selected symbol
                        for assignment based on alphabetical order
                        
        - None: Returns None if no unit clause is found
    """
    
    
    #List to store each symbol that is contained in each unit clause
    unit_clause_symbols= []
    
    #Iterating through all the clauses in the knowledge base
    for clause in KB:
        
        #Get all the unassigned symbols in the clause
        unassigned_symbols= [symbol for symbol in clause if symbol not in model]
        
        #If there is just one unassigned symbol (i.e. if the clause is a unit clause)
        if len(unassigned_symbols) == 1:
            
            #Store the symbol and its assignement that makes the clause True
            unassigned_symb= unassigned_symbols[0]
            unit_clause_symbols.append((unassigned_symb,unassigned_symb.sign))
         

    #If there is at least one unit clause
    if unit_clause_symbols:
        
        #If there are multiple unit clauses
        if len(unit_clause_symbols) > 1:
            
            
            #Return the symbol that comes first alphabetically & the assignment that makes the clause True
            return sorted(unit_clause_symbols)[0]
        
        #If there is just one unit clause symbol,return it & the assignment that makes it True
        else:
            return unit_clause_symbols[0]
     
    #If no unit clause is found, return None
    else:
        return None, None
        

def DPLL_Satisfiable(KB, heuristic_level=0):
    
    ''' satisfiable, model = DPLLSatisfiable(KB)
        Takes in a KB and returns whether the KB is satisfiable, and the model that makes it so
        
        KB: A knowledge base of clauses (CNF) consisting of a list of sets of literals.  A KB might look like
            [{A,B},{-A,C,D}]
        heuristic_level: An integer that will be passed in to specify which heuristics to implement 
            (only for the extension activities).
        satisfiable: Returns True if the KB is satisfiable, or False otherwise
        Model: A dictionary that assigns a truth value to each literal for the model that satisfies KB.
            For example, a model might look like {A: True, B: False}
    '''
    
    #A set containing all the propositional symbols in the KB
    symbols= reduce(lambda c1, c2: c1.union(c2), KB, set())
    
    result= DPLL(KB,symbols,{},heuristic_level,[])
    
    #If the model is unsatisfiable retuen False,None,0
    if result is False:
        return False, None, 0
    #If the model is satisfiable, return satisfiable, model, symbol_list
    else:
        return result

def DPLL(KB,symbols, model,heuristic_level,symbol_list):
    
    """
    Implement Davis-Putnam-Logemann-Loveland algorithm to check satisfiability of
    a knowledge base in propositional logic.
    
    
    Algorithm adapted from Figure 7.17 of Artificial Intelligence- A modern approach.


    Inputs:
        - KB: Knowledge base in the form of a list of sets of Literal objects, where each set
              represents a clause in CNF form.
        - symbols: A set containing all propositional symbols the KB
        - model(dict): The configuration of the world (assignment of literals) being tested to 
                       determine satisfiability of the KB. The keys of the dictionary represent the
                       Literals and the values of each represents their assignment in the model
                       (either True or False)
                 
    Outputs:
        - satisfiable (bool): Returns True if the KB is satisfiable. Returns False otherwise
        - model(dict): If the KB is satisfiable, returns a model (that was found) that 
                       satisfies the KB.
    
    """
    
    #Create list to store the clauses that cannot be determined from the a given model
    unknown_clauses= []
    
    #Initialize counter track the number of True clauses that result from a model's assignments
    num_true_clauses= 0
    
    #For each clause in the KB
    for clause in KB:
        
        #If the clause is False, return False
        if true_clause(clause,model) is False:
            return False
        
        #If the truth value of the clause cannot be determined, add to unknown_clauses
        elif true_clause(clause,model) is None:
            unknown_clauses.append(clause)
        
        #If the clause was proved True, update the counter
        else:
            num_true_clauses += 1
    
    #If all the clauses in the KB are True return satisfiable, model, symbol_list
    if num_true_clauses == len(KB):
        return True, model,symbol_list
    
    
    if heuristic_level==1:
        
        #Apply the DEGREE HEURISTIC
        current_symb, rest = degree_heuristic(symbols,unknown_clauses,model)
        
        #Add the symbol selected to the symbol_list
        symbol_list.append(current_symb)
        
        #Recursively call DPLL as the model and symbols (among other things) are updated
        return (DPLL(KB,set(rest), update(model,current_symb,True),heuristic_level,symbol_list) or 
                DPLL(KB,set(rest), update(model,current_symb,False),heuristic_level,symbol_list))
    
    
    elif heuristic_level==2:
        
        #Apply the PURE SYMBOL HEURISTIC
        current_symb, value = find_pure_symbol(symbols, unknown_clauses, model)
        
        #If a pure symbol was found
        if current_symb: 

            #Add it to the symbol_list to indicate assignment
            symbol_list.append(current_symb)

            #Get the rest of the symbols (after current_symb has been excluded)
            rest = symbols.copy()
            rest.remove(current_symb)

            #Recursively run the pure symbol heuristic assigning values to the pure symbols that
            #make them True in all the clauses, while updating the model and symbols
            return DPLL(KB, rest,update(model,current_symb,value),heuristic_level,symbol_list)
            
        #Apply the UNIT CLAUSE HEURISTIC
        current_symb, value = find_unit_clause(unknown_clauses, model)
        
        #If a unit clause was found
        if current_symb: 

            #Add the unit clause symbol to the symbol_list to indicate assignment
            symbol_list.append(current_symb)

            #Get the rest of the symbols (after current_symb has been excluded)
            rest = symbols.copy()
            rest.remove(current_symb)

            #Recursively run the unit clause heuristic to find unit clauses, and assign values them 
            #that make them True. This step is especially important given unit propagation
            #(assignment of one unit clause could create other unit clauses)
            return DPLL(KB, rest,update(model,current_symb,value),heuristic_level,symbol_list)
        
        
        #Apply the DEGREE HEURISTIC when no pure symbols or unit clauses are found
        current_symb, rest = degree_heuristic(symbols,unknown_clauses,model)
    
        
        #Add the selected symbol to the symbol list
        symbol_list.append(current_symb)
        
        #Recursively call DPLL on the updated model and symbol set
        return (DPLL(KB,set(rest), update(model,current_symb,True),heuristic_level,symbol_list) or 
                DPLL(KB,set(rest), update(model,current_symb,False),heuristic_level,symbol_list))
    
    
    elif heuristic_level== 3:
        
        #Apply the ADAPTED PURE SYMBOL HEURISTIC (DEGREE HEURISTIC TYPE ALTERATION)
        current_symb, value = find_pure_symbol(symbols, unknown_clauses, model, True)
        
        #If a pure symbol was found
        if current_symb: 

            #Add it to the symbol_list to indicate assignment
            symbol_list.append(current_symb)

            #Get the rest of the symbols (after current_symb has been excluded)
            rest = symbols.copy()
            rest.remove(current_symb)

            #Recursively run the pure symbol heuristic assigning values to the pure symbols that
            #make them True in all the clauses
            return DPLL(KB, rest,update(model,current_symb,value),heuristic_level,symbol_list)
        
        #Apply the UNIT CLAUSE HEURISTIC
        current_symb, value = find_unit_clause(unknown_clauses, model)
        
        #If a unit clause was found
        if current_symb: 

            #Add the unit clause symbol to the symbol_list to indicate assignment
            symbol_list.append(current_symb)

            #Get the rest of the symbols (after current_symb has been excluded)
            rest = symbols.copy()
            rest.remove(current_symb)

            #Recursively run the unit clause heuristic to find unit clauses, and assign values them 
            #that make them True. This step is especially important given unit propagation
            #(assignment of one unit clause could create other unit clauses)
            return DPLL(KB, rest,update(model,current_symb,value),heuristic_level,symbol_list)
        
        #Apply the DEGREE HEURISTIC when no pure symbol is found
        current_symb, rest = degree_heuristic(symbols,unknown_clauses,model)
        
        #Add the selected symbol to symbol_list
        symbol_list.append(current_symb)
        
        #Recursively call DPLL
        return (DPLL(KB,set(rest), update(model,current_symb,True),heuristic_level,symbol_list) or 
                DPLL(KB,set(rest), update(model,current_symb,False),heuristic_level,symbol_list))
        
    #If heuristic_level is 0 (BASIC FUNCTIONALITY TEST)
    else:
        
        #Select a symbol and exclude it from symbols to get rest
        current_symb, *rest = symbols
        
        #Add the selected symbol to the symbol_list
        symbol_list.append(current_symb)
        
        #Recursively call DPLL on the updated model and symbol set
        return (DPLL(KB,set(rest),update(model,current_symb,True),heuristic_level,symbol_list) or 
                DPLL(KB,set(rest),update(model,current_symb,False),heuristic_level,symbol_list))
    
    


<h1>Question 3</h1>

Implement your KB from Russell & Norvig in CNF by filling in the sets inside the list <code>KB</code> below.  Ensure that your KB is in a list of set format as stated in the assignment instructions.

- **S1: A $\Leftrightarrow$ (B $\lor$ E)**
    - [A $\Rightarrow$ (B $\lor$ E)] $\land$ [(B $\lor$ E) $\Rightarrow$ A]  (Biconditional Elimination)
    
    - ($\lnot$ A $\lor$ B $\lor$ E) $\land$ [$\lnot$(B $\lor$ E) $\lor$ A] (Implication Elimination)
    
    - ($\lnot$A $\lor$ B $\lor$ E) $\land$ [($\lnot$B $\land$ $\lnot$E) $\lor$ A] (De Morgan's Law)
    
    - ($\lnot$A $\lor$ B $\lor$ E) $\land$ ($\lnot$B $\lor$ A) $\land$ ($\lnot$E $\lor$ A) (Distributive law)
    
    
    
- **S2: E $\Rightarrow$ D**
    - $\lnot$E $\lor$ D (Implication Elimination)
    
    
- **S3: C $\land$ F $\Rightarrow$ $\lnot$ B**
    - $\lnot$(C $\land$ F) $\lor$ $\lnot$ B (Implication Elimination)
    
    - $\lnot$C $\lor$ $\lnot$F $\lor$ $\lnot$ B (De Morgan's Law)
    

- **S4: E $\Rightarrow$ B**
    - $\lnot$E $\lor$ B (Implication Elimination)
    
    
- **S5: B $\Rightarrow$ F**
    - $\lnot$B $\lor$ F (Implication Elimination)
    
    
- **S6: B $\Rightarrow$ C**
    - $\lnot$B $\lor$ C (Implication Elimination)

In [3]:
# Define some literals
A = Literal('A')
B = Literal('B')
C = Literal('C')
D = Literal('D')
E = Literal('E')
F = Literal('F')

#Translating the CNF sentences above into the KB, we get:
KB = [{-A,B,E},{-B,A},{-E,A},{-E,D},{-C,-F,-B},{-E,B},{-B,F},{-B,C}]

<h1>Extensions</h1>

1. Implement the degree heuristic for choosing symbols.  If <code>heuristic_level=1</code>, then the degree heuristic should be used to select which symbol to assign.
2. Implement the pure symbol and unit clause heuristics.  If <code>heuristic_level=2</code>, then these heuristics should be used to find select first a pure symbol, and if one is not found, then a unit clause.  If neither pure symbols nor unit clauses are found, then the degree heuristic should be used to select a symbol.  If there are multiple pure symbols or unit clauses found, then use alphabetical order to select amongst them.
3. Modify the pure symbol heuristic to choose the pure symbol that occurs in the most number of clauses.  This should be activated if <code>heuristic_level=3</code>.

<h1>Basic Functionality Tests</h1>

All of the tests in this section must be passed for the code to be considered fully functional.  Other unseen test cases will be used after submission to further verify functionality

In [4]:
# This section will test the correct definition of the literal class


# Test the name attribute works correctly
assert(A.name == 'A')

# Test that negation works correctly
assert(not (-C).sign)

In [5]:
# This section will test that the KB has been correctly converted to CNF by performing 
# tests over all possible models
import itertools
symbols = {A,B,C,D,E,F}
symbol_list = [A,B,C,D,E,F]

def evaluate_russell_norvig_KB(model):
    # Manually evaluate the KB using the model
    s = list()
    s.append(model[A] == (model[B] or model[E]))
    s.append(model[E] <= model[D])
    s.append((model[C] and model[F]) <= (not model[B]))
    s.append(model[E] <= model[B])
    s.append(model[B] <= model[F])
    s.append(model[B] <= model[C])
    return all(s)

def evaluate_KB(KB, model):
    model_true = True
    model_name_dict = {l.name: model[l] for l in model}
    for clause in KB:
        evaluation = {l.sign == model_name_dict[l.name] for l in clause if l.name in model_name_dict}
        model_true = model_true and any(evaluation)
    return model_true

all_models = list(itertools.product([False, True], repeat=6))
for i in range(0, len(all_models)):
    # Test all truth values
    test_model = {symbol_list[j]: all_models[i][j] for j in range(0,6)}
    
    # Test the model
    assert(evaluate_russell_norvig_KB(test_model) == evaluate_KB(KB, test_model))

In [6]:
# This section will test the basic working of DPLL
# Satisfiable model
test_KB = [{A, C},{-A, C}, {B, -C}]
t, model, symbol_trace = DPLL_Satisfiable(test_KB)
assert(evaluate_KB(test_KB, model))
assert(t)

# Unsatisfiable model
test_KB = [{A, C},{-A, C}, {-C}]
t, model, symbol_trace = DPLL_Satisfiable(test_KB)
assert(not t)

In [7]:
# This will test DPLL on the KB from Russell & Norvig
t, model, symbol_trace = DPLL_Satisfiable(KB,0)
print(model)

# This model is satisfiable.  Test that it is so
assert(t)

# Check that the model found actually works
assert(evaluate_KB(KB, model)) 

{B: False, F: True, C: True, A: False, E: False}


<h1>Extension Tests</h1>

This section will test that the degree heuristic, pure symbol and unit clause heuristics are correctly implemented.  Note that in your code, the <code>heuristic_level</code> needs to correctly activate the heuristic being tested, and the <code>symbol_list</code> needs to be correctly generated 

In [8]:
# Degree Heuristic Test for KB from Russell & Norvig
t, model, symbol_trace = DPLL_Satisfiable(KB,1)
print(model)

# This model is satisfiable.  Test that it is so
assert(t)

# Check that the model found actually works
assert(evaluate_KB(KB, model))  

# Test the symbol trace to ensure that the correct order is chosen
assert([l.name for l in symbol_trace] in [['B','F', 'E', 'C', 'C', 'E', 'A'], ['B', 'A', 'C', 'F', 'E', 'A']])

symbol_trace

{B: False, E: False, A: False}


[B, A, C, F, E, A]

In [9]:
# Pure Symbol & Unit Clause Heuristic Test for KB from Russell & Norvig
t, model, symbol_trace = DPLL_Satisfiable(KB,2)
print(model)


# This model is satisfiable.  Test that it is so
assert(t)

# Check that the model found actually works
assert(evaluate_KB(KB, model))

# Test the symbol trace to ensure that the correct order is chosen
assert([l.name for l in symbol_trace] == ['D', 'B', 'E', 'A', 'C', 'F', 'E', 'A'])

{D: True, B: False, E: False, A: False}


In [10]:
# Pure Symbol & Unit Clause Heuristic Test, choosing the most-frequently used pure symbol, for KB from Russell & Norvig
t, model, symbol_trace = DPLL_Satisfiable(KB,3)
print(model)
print(symbol_trace)

# This model is satisfiable.  Test that it is so
assert(t)

# Check that the model found actually works
assert(evaluate_KB(KB, model))

# Test the symbol trace to ensure that the correct order is chosen
assert([l.name for l in symbol_trace] == ['D', 'B', 'A', 'C', 'F', 'E', 'A'])

{D: True, B: False, E: False, A: False}
[D, B, A, C, F, E, A]
